Dieser Kurs basiert auf den [Langgraph Tutorials](https://github.com/langchain-ai/langgraph/tree/main/docs/docs/tutorials)

# 🚀 Vom Sprachmodell zum Chatbot

Mit modernen Sprachmodellen kann man sich fast wie mit einem Menschen unterhalten und sie können einem bei vielen Problemen helfen – etwa beim Programmieren oder Schreiben von Texten.
Sie haben ein umfangreiches Wissen aus allgemein verfügbaren Dokumenten, mit denen sie trainiert wurden.

Wenn es um sehr aktuelle Themen oder Fragen in einem speziellen Kontext geht, fehlen den Sprachmodellen die nötigen Informationen.
Mittlerweile gibt es die Möglichkeit, diese Informationen "nachzuliefer". Das nennt sich dann *Retrieval Augmented Generation*.

In diesem Tutorial werden wir einen Chatbot erstellen, der:

- [ ] Häufige Fragen durch Internetsuche beantworten kann.  
- [ ] Den Gesprächsfaden über Fragen hinweg aufrechterhält.  
- [ ] Fragen zu eigenen Dokumenten beantworten kann, etwa dem Koalitionsvertrag zwischen CDU und SPD oder den Werken von Shakespeare.

Dabei verwenden wir die Bibliothek [Langgraph](https://github.com/langchain-ai/langgraph), mit der das sehr einfach funktioniert.

Wir beginnen mit einem grundlegenden Chatbot und fügen schrittweise komplexere Funktionen hinzu. 
Lass uns loslegen!! 🌟

## Nötige Software installieren

Die folgenden beiden Zellen installieren ein paar benötigte Python-Bibliotheken und laden ein paar vertrauliche Konfigurationsdaten (API-Schlüssel): 

In [ ]:
%%capture --no-stderr
%pip install --upgrade --quiet dotenv langchain langgraph langsmith langchain-community langchain_chroma langchain_openai langchain-unstructured langchain-docling unstructured-client unstructured "unstructured[pdf]" python-magic

In [ ]:
%load_ext dotenv

In [ ]:
%dotenv /home/archive/FerienkursKI/.env

# Teil 1: Erstelle einen grundlegenden Chatbot

Zunächst werden wir einen einfachen Chatbot mit LangGraph erstellen. Dieser Chatbot wird direkt auf Benutzeranfragen antworten. 
Obwohl er einfach ist, veranschaulicht er die Kernkonzepte von LangGraph. Am Ende dieses Abschnitts wirst du einen rudimentären Chatbot erstellt haben.

Beginne mit der Erstellung eines `StateGraph`. Ein `StateGraph`-Objekt definiert die Struktur unseres Chatbots als "Zustandsmaschine". 
Wir werden Knoten hinzufügen, um das **LLM** (Large Language Model) und Funktionen darzustellen, die unser Chatbot aufrufen kann, sowie Kanten, um anzugeben, wie der Bot zwischen diesen Funktionen wechseln soll.

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

Unser Graph kann nun zwei wichtige Aufgaben erledigen:

1. Jeder Knoten kann den aktuellen Zustand als Eingabe empfangen und eine Aktualisierung des Zustands ausgeben.
2. Aktualisierungen der Nachrichten werden an die bestehende Liste angehängt, anstatt sie zu überschreiben. Dies geschieht dank der vorgefertigten `add_messages`-Funktion, die mit der annotierten Syntax verwendet wird.

Füge als Nächstes einen "chatbot"-Knoten hinzu. Knoten repräsentieren Verarbeitungsschritte und sind normalerweise reguläre Python-Funktionen.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

Beachte, dass die Funktion des Chatbot-Knotens den aktuellen Zustand als Eingabe akzeptiert und ein Dictionary zurückgibt, das eine aktualisierte Nachrichtenliste unter dem Schlüssel „messages“ enthält. Dies ist das grundlegende Muster für alle LangGraph-Knotenfunktionen.

Die add_messages-Funktion in unserem Zustand wird die Antwortmeldungen des LLM an die bereits im Zustand vorhandenen Nachrichten anhängen.

### Nächster Schritt: Einstiegspunkt hinzufügen
Um einen Einstiegspunkt für unseren Graphen zu erstellen, definieren wir eine Funktion, die als Startpunkt dient. Diese Funktion wird aufgerufen, wenn der Graph ausgeführt wird.

In [ ]:
graph_builder.add_edge(START, "chatbot")

Ebenso definieren wir einen Endpunkt. Dies weist den Graphen an: "Wann immer dieser Knoten ausgeführt wird, kannst du den Chatbot beenden."

In [ ]:
graph_builder.add_edge("chatbot", END)

Schließlich möchten wir in der Lage sein, unseren Graphen auszuführen. Dazu rufen wir `compile()` auf dem Graph-Builder auf. Dies erstellt einen **CompiledGraph**, den wir nutzen können, um unseren Zustand aufzurufen.

In [ ]:
graph = graph_builder.compile()

Du kannst den Graphen mit der Methode `get_graph` und einer der "draw"-Methoden visualisieren, wie z.B. `draw_ascii` oder `draw_png`. Jede der Zeichnen-Methoden benötigt zusätzliche Abhängigkeiten.

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

Jetzt lass uns den Chatbot ausführen!

**Tipp**: Du kannst die Chat-Schleife jederzeit beenden, indem du "quit", "exit" oder "q" eintippst.

In [ ]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

**Herzlichen Glückwunsch!** Du hast deinen ersten Chatbot mit LangGraph erstellt. Dieser Bot kann grundlegende Gespräche führen, indem er Benutzereingaben entgegennimmt und Antworten mit einem LLM generiert. Du kannst einen LangSmith Trace für den obigen Aufruf unter dem bereitgestellten Link einsehen.

Allerdings hast du möglicherweise bemerkt, dass das Wissen des Bots auf das beschränkt ist, was in seinen Trainingsdaten enthalten ist. Im nächsten Teil werden wir ein Websuchtool hinzufügen, um das Wissen des Bots zu erweitern und ihn leistungsfähiger zu machen.

# Teil 2: 🛠️ Verbesserung des Chatbots mit Werkzeugen

Um Anfragen zu bearbeiten, die unser Chatbot "aus dem Gedächtnis" nicht beantworten kann, 
werden wir ein Websuchtool integrieren. 
Unser Bot kann dieses Werkzeug nutzen, um relevante Informationen zu finden und bessere Antworten zu geben.

Wir benutzen dafür die [Suchmaschine Tavily](https://python.langchain.com/docs/integrations/tools/tavily_search/), die wir leicht als `Tool` integrieren können:

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(max_results=2)
tools = [tool]
tool.invoke("Wann wird Friedrich Merz zum Bundeskanzler gewählt?")

Als Nächstes erweitern wir unseren Graphen. 
Der folgende Code ist der gleiche wie in Teil 1, mit der Ausnahme, dass wir `bind_tools` zu unserem LLM hinzugefügt haben. 
Dies informiert das LLM über das korrekte JSON-Format, das verwendet werden soll, wenn es unsere Suchmaschine nutzen möchte.

In [ ]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


llm = ChatOpenAI(model="gpt-4o-mini")
# Modification: tell the LLM which tools it can call
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

Als Nächstes müssen wir eine Funktion erstellen, die die Werkzeuge tatsächlich ausführt, wenn sie aufgerufen werden. Dazu fügen wir die Werkzeuge einem neuen Knoten hinzu.

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile()

Die folgende Zelle zeigt den Aufbau des Chatbots:

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

Und mit dem folgenden Code kannst Du ihn ausprobieren:

In [ ]:
while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

# Teil 3: Ein "Gedächtnis" für den Chatbot

Unser Chatbot kann jetzt Werkzeuge verwenden, um Benutzerfragen zu beantworten, aber er erinnert sich nicht an den Kontext des Gesprächs. 
Dies schränkt seine Fähigkeit ein, kohärente, mehrteilige Gespräche zu führen.

LangGraph löst dieses Problem durch das Zwischenspeichern des Gesprächs, das *CheckPointing*. Wenn du einen Checkpointer beim Kompilieren des Graphen bereitstellst und eine `thread_id` beim Aufruf deines Graphen verwendest, speichert LangGraph automatisch den Zustand nach jedem Schritt. Wenn du den Graph erneut mit der gleichen `thread_id` aufrufst, lädt der Graph seinen gespeicherten Zustand, sodass der Chatbot dort fortfahren kann, wo er aufgehört hat.

Wir werden später sehen, dass Checkpointing viel leistungsfähiger ist als einfaches Chat-Gedächtnis – es ermöglicht dir, komplexe Zustände jederzeit für Fehlerwiederherstellung, menschliche Beteiligung in Arbeitsabläufen, Zeitreisen und mehr zu speichern und wiederherzustellen. Aber bevor wir uns zu weit vorausbegeben, lass uns Checkpointing hinzufügen, um mehrteilige Gespräche zu ermöglichen.

Um zu beginnen, erstelle einen `MemorySaver`-Checkpointer.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

Wir geben jetzt die Id der Unterhaltung als `thread_id` in der Konfiguration `config` mit.
Diese wird jeweils bei der Verarbeitung (`graph.stream()`) mitgegeben.

In [ ]:
config = {"configurable": {"thread_id": "1"}}

In [ ]:
def stream_graph_updates(user_input: str):
    events = graph.stream(
        {"messages": [{"role": "user", "content": user_input}]},
        config,
        stream_mode="values",
    )
    for event in events:
        event["messages"][-1].pretty_print()

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

# Teil 4: Abruf von Informationen

In diesem Abschnitt werden wir uns mit dem Retrieval (Abruf) von Informationen befassen, um die Fähigkeiten unseres Chatbots weiter auszubauen. Während unser Chatbot bereits in der Lage ist, grundlegende Antworten zu geben und auf Webanfragen zu reagieren, möchten wir ihm nun die Möglichkeit geben, gezielt Informationen aus einem `VectorStore` abzurufen. 

Durch die Integration von Retrieval-Techniken kann der Chatbot relevante Dokumente finden und präzise Antworten auf Benutzeranfragen geben, indem er verwertbare Informationen nutzt, die in den gespeicherten Daten gespeichert sind. 

## Was ist ein VectorStore?

Ein **VectorStore** ist wie ein großes, digitales Bücherregal für Informationen, das sich gut merken kann, was es gespeichert hat. Hier sind ein paar einfache Punkte, um es besser zu verstehen:

1. **Speicherort für Daten**: Stell dir vor, du hast viele verschiedene Bücher oder Dokumente mit Informationen. Ein VectorStore ist der Ort, an dem diese Informationen sicher gespeichert werden.

2. **Verstädnis von Bedeutungen**: Anstatt nur Worte zu speichern, "versteht" ein VectorStore die Bedeutung dieser Worte. Es wandelt Informationen in "Vektoren" um, die wie Punkte im Raum sind. Dadurch kann er ähnliche Informationen leicht finden.

3. **Schnelles Finden**: Wenn du etwas suchst, kann der VectorStore schnell die richtigen Informationen herausfinden, so wie ein Bibliothekar dir das richtige Buch in der Bibliothek zeigen würde.

4. **Helfer für Chatbots**: Für einen Chatbot bedeutet ein VectorStore, dass er bei Fragen auf eine große Sammlung von Wissen zugreifen kann. Wenn du zum Beispiel nach Informationen zu einem bestimmten Thema fragst, kann der Chatbot schnell die besten Antworten finden, weil er in diesem speziellen Speicher nachsehen kann.

Zusammengefasst: Ein VectorStore ist ein smarter Speicher für Informationen, der es einem Chatbot ermöglicht, schnell Antworten zu finden und dir bei deinen Fragen zu helfen!

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import chromadb

## Laden des `VectorStore`

Die "Befüllung" des `VectorStore` ist in das Notebook [create_chroma](./chreate_chroma.ipynb) ausgelagert, da dieser Vorgang länger dauert (und Geld kostet, da wir die Vektoren zu den Dokumenten mithilfe von OpenAI erstellen).

Hier laden wir die von dort erstellte Vektordatenbank:


In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
chromadb.configure(allow_reset=True)

client = chromadb.PersistentClient(path="./chroma")

vector_store = Chroma(
    client=client,
    collection_name="Koalitionsvertrag",
    embedding_function=embeddings,
)

retriever = vector_store.as_retriever()

## Dokumentensuche als Werkzeug

Wir fügen die Suche im Koalitionsvertrag jetzt wie die Websuche als Tool hinzu:

In [ ]:
from langchain_core.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults

retriever_tool = create_retriever_tool(retriever, "Koalitionsvertrag", "Durchsuche dem Koalitionsvertrag von CDU und SPD")
search_tool = TavilySearchResults(max_results=2)
tools = [search_tool, retriever_tool]

Wie schon vorher bauen die Werzeuge in den Chatbot-Graphen ein:

In [ ]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


llm = ChatOpenAI(model="gpt-4o-mini")
# Modification: tell the LLM which tools it can call
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)


tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

Der folgende Code erlaubt uns, den Chatbot zu testen:

In [26]:
config = {"configurable": {"thread_id": "1"}}

def stream_graph_updates(user_input: str):
    events = graph.stream(
        {"messages": [{"role": "user", "content": user_input}]},
        config,
        stream_mode="values",
    )
    for event in events:
        event["messages"][-1].pretty_print()

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

User: What do you know about LangGraph?
================================ Human Message =================================

What do you know about LangGraph?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_bzqVd0hCWmUueMPBnvD0NHlx)
 Call ID: call_bzqVd0hCWmUueMPBnvD0NHlx
  Args:
    query: LangGraph
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "LangGraph Quickstart - GitHub Pages", "url": "https://langchain-ai.github.io/langgraph/tutorials/introduction/", "content": "[](https://langchain-ai.github.io/langgraph/tutorials/introduction/#__codelineno-9-1)Assistant: LangGraph is a library designed to help build stateful multi-agent applications using language models. It provides tools for creating workflows and state machines to coordinate multiple AI agents or language model interactions. LangGraph is built on top of LangChain, lever